In [96]:
from nbdev.export import *
notebook2script()

Converted config.ipynb.
Converted index.ipynb.
Converted rest.ipynb.
Converted services.ipynb.
Converted term.ipynb.


In [11]:
# default_exp services
# export
from pathlib import Path
from config import Config
import yaml
import os
import time

In [80]:
# export

quick_scrum_folder = 'quick-scrum'

def validate_project(path):
    required = ['project.yaml', 'sandbox.store', 'backlog.store', 'users','docs']
    return all([(f/quick_scrum_folder/name).exists() for name in required])

def scan_for_projects(path):
    project_folders = [f.parent.absolute() for f in path.glob('**/{quick_scrum_folder}')]
    return [f for f in project_folders if validate_project(f)]
        
def find_project(path):
    qs_path = path/'quick-scrum'
    if qs_path.exists():
        return qs_path
    elif path.parent == path:
        return None
    else:
        return find_project(path.parent)

In [95]:
# export

class Project:
    config = Config()
    project_folder = 'qs-proj'
    
    def __init__(self, path: Path):
        self.path = path
        if self.exists():
            self.config.projects[self.path.name] = self.path            
        
    @staticmethod
    def find(path):
        print('hello')
        if (path/Project.project_folder).exists():
            return Project(path)
        elif path.name == '':
            return None
        else:
            return None # Project.find(path.parent)
        
    def exists(self):
        return (self.path/self.project_folder).exists()

    def init(self, project):
        existing_projects = scan_for_projects(self.path)
        if len(existing_projects):
            paths = ','.join([str(p) for p in existing_projects])
            raise FileExistsError(f'A quick-scrum project already exists in {paths}')

        required = ['sandbox.store', 'backlog.store', 'docs','users']
        [(self.path/self.project_folder/n).mkdir(parents=True) for n in required]
        with (self.path/self.project_folder/'project.yaml').open('w') as f:
            c = {**project, 'creation_timestamp': time.time()}
            yaml.dump(c, f)
        self.config.projects[self.path.name] = self.path      
        
    def list_stores(self):
        pf = self.path/self.project_folder
        return [f.stem for f in pf.iterdir() if f.is_dir() and f.suffix == '.store']
        
    def get_store(self, store_name):
        sf = self.path/self.project_folder/f'{store_name}.store'
        return Store(sf) if sf.exists() else None
    
    def new_store(self, store_name):
        sf = self.path/self.project_folder/f'{store_name}.store'
        if sf.exists():
            raise AttributeError(f'Store {store_name} already exists')
        else:
            sf.mkdir()
            return Store(sf)
        
    def list_sprints(self):
        "Get all the sprints in the project prj"
        return [n for n in self.list_stores() if n.startswith('sprint-')]
    
    def new_sprint(self):
        "Create a new sprint"
        ss = sorted(self.list_sprints())
        if ss:
            last = ss[-1]
            (_,n) = last.split('-')
            s = f'sprint-{int(n)+1}'
        else:
            s = f'sprint-1'
        return self.new_store(s)


In [84]:
p = Project(Path('../data/test5'))
s = p.new_sprint()

In [70]:
s['1 - User Creation'] = {'test':3}

In [71]:
s.keys()

['~1 - User Creation']

In [78]:
# export
class Store:
    def __init__(self, path):
        self.path = path
    
    
    def __getitem__(self, story_id):
        "Get the content of a story"
        p = self.path/f'{story_id}.yaml'
        if not p.exists():
            return None
        with p.open() as f:
            c = yaml.load(f, Loader=yaml.FullLoader)
            return {'name': story_name, **c}
    
    def __setitem__(self, story_id, story):
        p = self.path/f'{story_id}.yaml'
        with p.open('w') as f:
            yaml.dump(story, f)
        return p.stem

    def __delitem__(self, story_id):
        p = self.path/f'{story_name}.yaml'
        if p.exists():
            p.unlink()
        else:
            raise KeyError(story_id)
    
    def keys(self):
        return [f.stem for f in self.path.iterdir() if f.suffix == '.yaml']
    
    

In [15]:
# export
class Scrum:
    def __init__(self, home_folder):
        self.home = Path(home_folder)
    
    def get_projects(self):
        "Get all the projects available in the home folder"
        return [p.name for p in (self.home/'projects').iterdir() if p.is_dir()]
    
    def get_project(self, project_name):
        "Get the project info"
        p = self.home/'projects'/project_name/'project.yaml'
        if p.exists():
            with p.open() as f:
                return yaml.load(f, Loader=yaml.FullLoader)
        else:
            return None
    
    def new_project(self, project):
        n = project['name']
        p = self.home/'projects'/n
        if p.exists():
            raise ValueError(f'Project {n} already exist')
        
        [f.mkdir(parents=True) for f in [p, p/'backlog', p/'sandbox']]
        with (p/'project.yaml').open('w') as f:
            c = {**project, 'created': time.time()}
            yaml.dump(c, f)
        
    def get_sprints(self, project_name):
        "Get all the sprints in the project prj"
        return [p.name for p in (self.home/'projects'/project_name)
                .glob('sprint-*') if p.is_dir()]
        
    def new_sprint(self, project_name):
        "Create a new sprint"
        ss = sorted(self.get_sprints(project_name))
        if ss:
            last = ss[-1]
            (_,n) = last.split('-')
            s = f'sprint-{int(n)+1}'
        else:
            s = f'sprint-1'
        (self.home/'projects'/project_name/s).mkdir()
        return s
        
    def get_stories(self, project_name, store):
        "Get all the stories name for a store, i.e. backlog, sprint..."
        pp = self.home/'projects'/project_name/store
        pp = sorted(pp.glob('*.yaml'), key=os.path.getmtime)
        return [p.stem for p in pp]

    def get_story(self, project_name, store, story_name):
        "Get the content of a story"
        p = self.home/'projects'/project_name/store/f'{story_name}.yaml'
        if not p.exists():
            return None
        with p.open() as f:
            c = yaml.load(f, Loader=yaml.FullLoader)
            return {'name': story_name, **c}

    def del_story(self, project_name, store, story_name):
        p = self.home/'projects'/project_name/store/f'{story_name}.yaml'
        if p.exists():
            p.unlink()
            return True
        else:
            return False
        
    def set_story(self, project_name, store, story):
        name = story['name']
        p = self.home/'projects'/project_name/store/f'~{name}.yaml'
        with p.open('w') as f:
            yaml.dump(story, f)
        p = p.replace(self.home/'projects'/project_name/store/f'{name}.yaml')
        return p.stem
        
    def touch_story(self, project_name, store, story):
        name = story['name']
        p = self.home/'projects'/project_name/store/f'~{name}.yaml'
        p.touch()
        
    def move_story(self, project_name, source, target, story_name):
        "Move a story to a different container"
        p = self.home/'projects'/project_name/source/f'{story_name}.yaml'
        if p.exist():
            p.replace(self.home/'projects'/project_name/target/f'{name}.yaml')
            return True
        else:
            return False

In [9]:
s = Scrum('../data')

In [10]:
s.get_projects()

['new Proj 34h', 'new Proj', 'new Proj 34', 'new Proj 3', 'test-prj']

In [12]:
s.new_project({'name':'test-prj'})

ValueError: Project test-prj already exist

In [7]:
s.new_sprint('test-prj')

'sprint-7'

In [8]:
s.get_stories('test-prj','sprint-2')

[]

In [27]:
s.get_logs('te2st-prj')

[]

In [ ]:
st = s.get_story('test-prj','sandbox','define-the-interface')

In [ ]:
s.set_story('test-prj','sandbox',{
    'name': 'a new story',
    'description': 'with some description'
})

In [ ]:
s = [2]

In [ ]:
if s: print('dd')